# Kayak Hotels and Weather data - ETL Process


### Imports

In [7]:
import pandas as pd
import os
import re
import psycopg2
import boto3

from sqlalchemy import create_engine

from dotenv import load_dotenv
load_dotenv()

True

## ETL (Extract, Transform, Load)

### Extract Raw Datasets

In [10]:
engine = create_engine(os.environ["DBURI"], pool_pre_ping=True)

# S3 files 
HOTELS_S3  = "s3://kayak-weather-hotel-data/Hotel_dataset_final_cityID.csv"
WEATHER_S3 = "s3://kayak-weather-hotel-data/Weather_dataset_final_cityID.csv"

# Build explicit storage options from env vars 
STORAGE = {
    "key": os.environ["AWS_ACCESS_KEY_ID"],
    "secret": os.environ["AWS_SECRET_ACCESS_KEY"],
    "client_kwargs": {"region_name": os.getenv("AWS_DEFAULT_REGION", "eu-west-3")},
}

hotels  = pd.read_csv(HOTELS_S3,  storage_options=STORAGE)
weather = pd.read_csv(WEATHER_S3, storage_options=STORAGE)

print("Hotels columns:", hotels.columns.tolist())
print("Weather columns:", weather.columns.tolist())


Hotels columns: ['City', 'Hotel Name', 'Rate', 'Number of Reviews', 'Neighborhood', 'Distance from Center', 'Description', 'Facilities', 'Address', 'Latitude', 'Longitude', 'Hotel URL', 'City_ID']
Weather columns: ['dt', 'sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase', 'summary', 'temp', 'feels_like', 'pressure', 'humidity', 'dew_point', 'wind_speed', 'wind_deg', 'wind_gust', 'weather', 'clouds', 'pop', 'uvi', 'rain', 'City', 'Latitude', 'Longitude', 'snow', 'day', 'min', 'max', 'night', 'eve', 'morn', 'average_min_temp', 'average_max_temp', 'rain_fall', 'avg_clouds', 'City_ID']


In [14]:
hotels.head()

,City,Hotel Name,Rate,Number of Reviews,Neighborhood,Distance from Center,Description,Facilities,Address,Latitude,Longitude,Hotel URL,City_ID
0,Aix en Provence,Renaissance Aix-en-Provence Hotel,"Scored 8.5\n8.5\nVery Good\n1,692 reviews",1692.0,Aix-en-Provence,0.6 km from downtown,Located within a 10-minute walk of the center ...,"2 swimming pools, Private Parking, Spa, Free W...",Aix-en-Provence,43.526240,5.438205,https://www.booking.com/hotel/fr/renaissance-a...,33
1,Aix en Provence,Aquabella Hôtel & Spa,"Scored 8.6\n8.6\nExcellent\n1,485 reviews",1485.0,"Aix-en-Provence Historic Centre, Aix-en-Provence",0.6 km from downtown,"Surrounded by a beautiful garden, Aquabella of...","Outdoor swimming pool, Parking, Spa, Restauran...","Aix-en-Provence Historic Centre, Aix-en-Provence",43.531264,5.445070,https://www.booking.com/hotel/fr/hotel-aquabel...,33
2,Aix en Provence,Hôtel Boutique Cézanne centre Aix-en-Provence,"Scored 8.7\n8.7\nExcellent\n1,726 reviews",1726.0,Aix-en-Provence,300 m from downtown,This boutique design hotel is located in the c...,"Private Parking, Free Wifi, Family rooms, Non-...",Aix-en-Provence,43.523562,5.445937,https://www.booking.com/hotel/fr/cezanne-aix-e...,33
3,Aix en Provence,Hôtel des Augustins,"Scored 8.4\n8.4\nVery Good\n2,246 reviews",2246.0,"Aix-en-Provence Historic Centre, Aix-en-Provence",150 m from downtown,Located just of the famous Cours Mirabeau in t...,"Parking, Free Wifi, Family rooms, Non-smoking ...","Aix-en-Provence Historic Centre, Aix-en-Provence",43.526887,5.447271,https://www.booking.com/hotel/fr/les-augustins...,33
4,Aix en Provence,Grand Hôtel Roi René Aix - MGallery Collection,"Scored 8.4\n8.4\nVery Good\n1,099 reviews",1099.0,Aix-en-Provence,500 m from downtown,Located just a 5-minute walk from Cours Mirabe...,"Outdoor swimming pool, Private Parking, Free W...",Aix-en-Provence,43.523325,5.449726,https://www.booking.com/hotel/fr/grand-mercure...,33


In [15]:
weather.head()

,dt,sunrise,sunset,moonrise,moonset,moon_phase,summary,temp,feels_like,pressure,...,min,max,night,eve,morn,average_min_temp,average_max_temp,rain_fall,avg_clouds,City_ID
0,2025-02-03 12:00:00+00:00,1738567966,1738602416,1738575420,0,0.18,There will be clear sky today,"{'day': 7.86, 'min': 1.21, 'max': 8.62, 'night...","{'day': 6.09, 'night': -0.26, 'eve': 5.08, 'mo...",1028,...,1.21,8.62,2.94,7.09,1.26,2.8625,8.745,0.976667,55.875,276
1,2025-02-04 12:00:00+00:00,1738654281,1738688914,1738662900,1738626480,0.22,Expect a day of partly cloudy with rain,"{'day': 9.43, 'min': 1.32, 'max': 11.07, 'nigh...","{'day': 6.36, 'night': 6.91, 'eve': 5.43, 'mor...",1027,...,1.32,11.07,8.90,7.98,1.32,2.8625,8.745,0.976667,55.875,276
2,2025-02-05 12:00:00+00:00,1738740593,1738775413,1738750680,1738717980,0.25,Expect a day of partly cloudy with clear spells,"{'day': 9.74, 'min': 4.7, 'max': 10.13, 'night...","{'day': 8.66, 'night': 2.12, 'eve': 4.17, 'mor...",1040,...,4.70,10.13,4.70,6.47,5.05,2.8625,8.745,0.976667,55.875,276
3,2025-02-06 12:00:00+00:00,1738826905,1738861912,1738838880,1738809480,0.29,"You can expect partly cloudy in the morning, w...","{'day': 7.97, 'min': 3.76, 'max': 8.47, 'night...","{'day': 4.51, 'night': -0.41, 'eve': 0.61, 'mo...",1039,...,3.76,8.47,4.29,5.01,4.13,2.8625,8.745,0.976667,55.875,276
4,2025-02-07 12:00:00+00:00,1738913214,1738948412,1738927680,1738900680,0.33,"There will be clear sky until morning, then pa...","{'day': 4.6, 'min': 2.04, 'max': 4.6, 'night':...","{'day': 0.21, 'night': -1.53, 'eve': -0.93, 'm...",1026,...,2.04,4.60,2.56,3.35,2.04,2.8625,8.745,0.976667,55.875,276


### Transform Hotels Dataset

In [11]:
## Hotels data
hotels_clean = hotels.copy()

# Extract numeric rate from full reviews column 
hotels_clean["Clean_Rate"] = hotels_clean["Rate"].str.extract(r'(\d+\.\d+)').astype(float)

# Drop original rate column
hotels_clean = hotels_clean.drop(columns=["Rate"])

# Keep num reviews col as integer (no decimal place)
hotels_clean["Number of Reviews"] = hotels_clean["Number of Reviews"].astype(int)

# Replace spaces with underscores in all column names
hotels_clean.columns = [col.replace(" ", "_") for col in hotels_clean.columns]

hotels_clean.head()


,City,Hotel_Name,Number_of_Reviews,Neighborhood,Distance_from_Center,Description,Facilities,Address,Latitude,Longitude,Hotel_URL,City_ID,Clean_Rate
0,Aix en Provence,Renaissance Aix-en-Provence Hotel,1692,Aix-en-Provence,0.6 km from downtown,Located within a 10-minute walk of the center ...,"2 swimming pools, Private Parking, Spa, Free W...",Aix-en-Provence,43.526240,5.438205,https://www.booking.com/hotel/fr/renaissance-a...,33,8.5
1,Aix en Provence,Aquabella Hôtel & Spa,1485,"Aix-en-Provence Historic Centre, Aix-en-Provence",0.6 km from downtown,"Surrounded by a beautiful garden, Aquabella of...","Outdoor swimming pool, Parking, Spa, Restauran...","Aix-en-Provence Historic Centre, Aix-en-Provence",43.531264,5.445070,https://www.booking.com/hotel/fr/hotel-aquabel...,33,8.6
2,Aix en Provence,Hôtel Boutique Cézanne centre Aix-en-Provence,1726,Aix-en-Provence,300 m from downtown,This boutique design hotel is located in the c...,"Private Parking, Free Wifi, Family rooms, Non-...",Aix-en-Provence,43.523562,5.445937,https://www.booking.com/hotel/fr/cezanne-aix-e...,33,8.7
3,Aix en Provence,Hôtel des Augustins,2246,"Aix-en-Provence Historic Centre, Aix-en-Provence",150 m from downtown,Located just of the famous Cours Mirabeau in t...,"Parking, Free Wifi, Family rooms, Non-smoking ...","Aix-en-Provence Historic Centre, Aix-en-Provence",43.526887,5.447271,https://www.booking.com/hotel/fr/les-augustins...,33,8.4
4,Aix en Provence,Grand Hôtel Roi René Aix - MGallery Collection,1099,Aix-en-Provence,500 m from downtown,Located just a 5-minute walk from Cours Mirabe...,"Outdoor swimming pool, Private Parking, Free W...",Aix-en-Provence,43.523325,5.449726,https://www.booking.com/hotel/fr/grand-mercure...,33,8.4


In [47]:
hotels_clean.to_csv("hotels_clean.csv", index=False)

### Load Cleaned Hotels Dataset

In [12]:
import pandas as pd
from sqlalchemy import create_engine

# Credentials
USER = "jedha_user"
DBPASS = os.getenv("DBPASS")
DBHOST = os.getenv("DBHOST") 
DBNAME = "jedha-kayak"

# 1) Create SQLAlchemy engine
engine = create_engine(
    f"postgresql+psycopg2://{USER}:{DBPASS}@{DBHOST}/{DBNAME}",
    echo=True  # logs SQL for debugging
)

# 2) Load your cleaned dataframe into the DW
hotels_clean.to_sql(
    "hotel_clean_data",   # table name in DB
    engine,
    if_exists="replace",  
    index=False          
)

print("✅ Data loaded successfully into Postgres!")


2025-08-19 22:23:42,682 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-08-19 22:23:42,684 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-19 22:23:42,712 INFO sqlalchemy.engine.Engine select current_schema()
2025-08-19 22:23:42,712 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-19 22:23:42,729 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-08-19 22:23:42,730 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-19 22:23:42,746 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-19 22:23:42,780 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

### Transform Weather Dataset

In [22]:
## Weather data

# Show all columns
pd.set_option("display.max_columns", None)

weather.head()

,dt,sunrise,sunset,moonrise,moonset,moon_phase,summary,temp,feels_like,pressure,humidity,dew_point,wind_speed,wind_deg,wind_gust,weather,clouds,pop,uvi,rain,City,Latitude,Longitude,snow,day,min,max,night,eve,morn,average_min_temp,average_max_temp,rain_fall,avg_clouds,City_ID
0,2025-02-03 12:00:00+00:00,1738567966,1738602416,1738575420,0,0.18,There will be clear sky today,"{'day': 7.86, 'min': 1.21, 'max': 8.62, 'night...","{'day': 6.09, 'night': -0.26, 'eve': 5.08, 'mo...",1028,75,3.71,3.51,167,4.14,"[{'id': 800, 'main': 'Clear', 'description': '...",4,0.00,1.37,NaN,Mont Saint Michel,48.64,-1.51,NaN,7.86,1.21,8.62,2.94,7.09,1.26,2.8625,8.745,0.976667,55.875,276
1,2025-02-04 12:00:00+00:00,1738654281,1738688914,1738662900,1738626480,0.22,Expect a day of partly cloudy with rain,"{'day': 9.43, 'min': 1.32, 'max': 11.07, 'nigh...","{'day': 6.36, 'night': 6.91, 'eve': 5.43, 'mor...",1027,81,6.13,6.95,209,10.61,"[{'id': 500, 'main': 'Rain', 'description': 'l...",1,1.00,1.30,0.59,Mont Saint Michel,48.64,-1.51,NaN,9.43,1.32,11.07,8.90,7.98,1.32,2.8625,8.745,0.976667,55.875,276
2,2025-02-05 12:00:00+00:00,1738740593,1738775413,1738750680,1738717980,0.25,Expect a day of partly cloudy with clear spells,"{'day': 9.74, 'min': 4.7, 'max': 10.13, 'night...","{'day': 8.66, 'night': 2.12, 'eve': 4.17, 'mor...",1040,70,4.26,3.56,351,5.74,"[{'id': 803, 'main': 'Clouds', 'description': ...",70,0.76,1.24,NaN,Mont Saint Michel,48.64,-1.51,NaN,9.74,4.70,10.13,4.70,6.47,5.05,2.8625,8.745,0.976667,55.875,276
3,2025-02-06 12:00:00+00:00,1738826905,1738861912,1738838880,1738809480,0.29,"You can expect partly cloudy in the morning, w...","{'day': 7.97, 'min': 3.76, 'max': 8.47, 'night...","{'day': 4.51, 'night': -0.41, 'eve': 0.61, 'mo...",1039,67,2.13,7.03,47,13.51,"[{'id': 803, 'main': 'Clouds', 'description': ...",55,0.00,1.09,NaN,Mont Saint Michel,48.64,-1.51,NaN,7.97,3.76,8.47,4.29,5.01,4.13,2.8625,8.745,0.976667,55.875,276
4,2025-02-07 12:00:00+00:00,1738913214,1738948412,1738927680,1738900680,0.33,"There will be clear sky until morning, then pa...","{'day': 4.6, 'min': 2.04, 'max': 4.6, 'night':...","{'day': 0.21, 'night': -1.53, 'eve': -0.93, 'm...",1026,72,-0.21,6.84,48,12.85,"[{'id': 803, 'main': 'Clouds', 'description': ...",83,0.00,1.37,NaN,Mont Saint Michel,48.64,-1.51,NaN,4.60,2.04,4.60,2.56,3.35,2.04,2.8625,8.745,0.976667,55.875,276


In [13]:
weather_clean = weather.copy()

# Convert date column (object) to datetime for easier manipulation
weather_clean["dt"] = pd.to_datetime(weather_clean["dt"], errors="coerce", dayfirst=True)
weather_clean = weather_clean.rename(columns={"dt": "Date"})


# Keep only columns of interest
keep_cols = [
    "Date", "City_ID", "City", "Latitude", "Longitude",
    "summary", 
    #"weather",
    #"temp", "feels_like", #already extracted previously
    "min", "max",
    "average_min_temp", "average_max_temp",
    "pressure", "humidity", "dew_point",
    "wind_speed", "wind_deg", "wind_gust",
    "clouds", "avg_clouds",
    "pop", "rain", "rain_fall", "snow",
    "uvi"
]

# Keep only those columns that exist in df
weather_clean = weather_clean[[col for col in keep_cols if col in weather_clean.columns]].copy()

# Capitalize first letter of each column name
def clean_col(col):
    col = col.replace("_", " ").title().replace(" ", "_")
    # Fix common acronyms
    col = col.replace("Id", "ID").replace("Uvi", "UVI")
    return col

weather_clean.columns = [clean_col(c) for c in weather_clean.columns]

weather_clean.head()


,Date,City_ID,City,Latitude,Longitude,Summary,Min,Max,Average_Min_Temp,Average_Max_Temp,...,Wind_Speed,Wind_Deg,Wind_Gust,Clouds,Avg_Clouds,Pop,Rain,Rain_Fall,Snow,UVI
0,2025-03-02 12:00:00+00:00,276,Mont Saint Michel,48.64,-1.51,There will be clear sky today,1.21,8.62,2.8625,8.745,...,3.51,167,4.14,4,55.875,0.00,NaN,0.976667,NaN,1.37
1,2025-04-02 12:00:00+00:00,276,Mont Saint Michel,48.64,-1.51,Expect a day of partly cloudy with rain,1.32,11.07,2.8625,8.745,...,6.95,209,10.61,1,55.875,1.00,0.59,0.976667,NaN,1.30
2,2025-05-02 12:00:00+00:00,276,Mont Saint Michel,48.64,-1.51,Expect a day of partly cloudy with clear spells,4.70,10.13,2.8625,8.745,...,3.56,351,5.74,70,55.875,0.76,NaN,0.976667,NaN,1.24
3,2025-06-02 12:00:00+00:00,276,Mont Saint Michel,48.64,-1.51,"You can expect partly cloudy in the morning, w...",3.76,8.47,2.8625,8.745,...,7.03,47,13.51,55,55.875,0.00,NaN,0.976667,NaN,1.09
4,2025-07-02 12:00:00+00:00,276,Mont Saint Michel,48.64,-1.51,"There will be clear sky until morning, then pa...",2.04,4.60,2.8625,8.745,...,6.84,48,12.85,83,55.875,0.00,NaN,0.976667,NaN,1.37


### Load Cleaned Weather Dataset

In [14]:
#Load cleaned dataframe into the DW
weather_clean.to_sql(
    "weather_clean_data",   # table name in DB
    engine,
    if_exists="replace",  
    index=False          
)

print("✅ Data loaded successfully into Postgres!")

2025-08-19 22:27:02,563 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-19 22:27:02,676 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-08-19 22:27:02,678 INFO sqlalchemy.engine.Engine [cached since 199.9s ago] {'table_name': 'weather_clean_data', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-08-19 22:27:02,709 INFO sqlalchemy.engine.Engine 
CREATE TABLE weather_clean_data (
	"Date" TIMESTAMP WITH TIME ZONE, 
	"City_ID" BIGINT, 
	"City" TEXT, 
	"Latitude" FLOAT(53), 
	"Longitude" FLOAT(53), 
	"Summary" 

### Merge Clean Hotels + Weather Datasets

In [ ]:
# Merge the two datasets by City_ID

hotel_weather_data = pd.merge(
    hotels_clean,
    weather_clean,
    on="City_ID",
    how="inner"  #only keep rows with matching City_IDs
)

print("Merged dataset shape:", hotel_weather_data.shape)

Merged dataset shape: (1000, 35)


In [37]:
hotel_weather_data.head()

,City_x,Hotel_Name,Number_of_Reviews,Neighborhood,Distance_from_Center,Description,Facilities,Address,Latitude_x,Longitude_x,Hotel_URL,City_ID,Clean_Rate,Date,City_y,Latitude_y,Longitude_y,Summary,Min,Max,Average_Min_Temp,Average_Max_Temp,Pressure,Humidity,Dew_Point,Wind_Speed,Wind_Deg,Wind_Gust,Clouds,Avg_Clouds,Pop,Rain,Rain_Fall,Snow,UVI
0,Aix en Provence,Renaissance Aix-en-Provence Hotel,1692,Aix-en-Provence,0.6 km from downtown,Located within a 10-minute walk of the center ...,"2 swimming pools, Private Parking, Spa, Free W...",Aix-en-Provence,43.52624,5.438205,https://www.booking.com/hotel/fr/renaissance-a...,33,8.5,2025-03-02 11:00:00+00:00,Aix en Provence,43.53,5.45,"There will be partly cloudy until morning, the...",6.20,14.63,6.76,12.96125,1022,48,1.00,1.96,302,2.68,0,54.25,0.00,NaN,0.595,NaN,1.97
1,Aix en Provence,Renaissance Aix-en-Provence Hotel,1692,Aix-en-Provence,0.6 km from downtown,Located within a 10-minute walk of the center ...,"2 swimming pools, Private Parking, Spa, Free W...",Aix-en-Provence,43.52624,5.438205,https://www.booking.com/hotel/fr/renaissance-a...,33,8.5,2025-04-02 11:00:00+00:00,Aix en Provence,43.53,5.45,Expect a day of partly cloudy with clear spells,6.24,13.58,6.76,12.96125,1026,38,-1.77,2.31,234,2.17,41,54.25,0.00,NaN,0.595,NaN,1.79
2,Aix en Provence,Renaissance Aix-en-Provence Hotel,1692,Aix-en-Provence,0.6 km from downtown,Located within a 10-minute walk of the center ...,"2 swimming pools, Private Parking, Spa, Free W...",Aix-en-Provence,43.52624,5.438205,https://www.booking.com/hotel/fr/renaissance-a...,33,8.5,2025-05-02 11:00:00+00:00,Aix en Provence,43.53,5.45,Expect a day of partly cloudy with clear spells,5.80,13.91,6.76,12.96125,1031,43,-0.97,1.72,290,2.02,0,54.25,0.00,NaN,0.595,NaN,1.88
3,Aix en Provence,Renaissance Aix-en-Provence Hotel,1692,Aix-en-Provence,0.6 km from downtown,Located within a 10-minute walk of the center ...,"2 swimming pools, Private Parking, Spa, Free W...",Aix-en-Provence,43.52624,5.438205,https://www.booking.com/hotel/fr/renaissance-a...,33,8.5,2025-06-02 11:00:00+00:00,Aix en Provence,43.53,5.45,There will be clear sky today,5.99,13.79,6.76,12.96125,1028,41,-0.70,2.60,245,2.35,0,54.25,0.00,NaN,0.595,NaN,1.97
4,Aix en Provence,Renaissance Aix-en-Provence Hotel,1692,Aix-en-Provence,0.6 km from downtown,Located within a 10-minute walk of the center ...,"2 swimming pools, Private Parking, Spa, Free W...",Aix-en-Provence,43.52624,5.438205,https://www.booking.com/hotel/fr/renaissance-a...,33,8.5,2025-07-02 11:00:00+00:00,Aix en Provence,43.53,5.45,"There will be clear sky until morning, then pa...",5.84,10.21,6.76,12.96125,1022,64,3.47,7.53,107,12.79,96,54.25,0.02,NaN,0.595,NaN,1.74


### Tansform Merged Dataset

In [16]:
# Final cleaning

## Drop duplicate city/location columns 
cols_to_drop = ["City_y", "Latitude_y", "Longitude_y"]
hotel_weather_data = hotel_weather_data.drop(columns=[c for c in cols_to_drop if c in hotel_weather_data.columns])

## Rename _x / _y leftovers
hotel_weather_data = hotel_weather_data.rename(columns={
    "City_x": "City",
    "Latitude_x": "Latitude",
    "Longitude_x": "Longitude",
    "Clean_Rate": "Rate",
    "Date": "Weather_Date",
    "Summary": "Weather_Summary",
    "Min": "Min_Temp",
    "Max": "Max_Temp"
})

# Ensure City_ID is first
cols = ["City_ID"] + [c for c in hotel_weather_data.columns if c != "City_ID"]
hotel_weather_data = hotel_weather_data[cols]

hotel_weather_data.head()

,City_ID,City,Hotel_Name,Number_of_Reviews,Neighborhood,Distance_from_Center,Description,Facilities,Address,Latitude,...,Wind_Speed,Wind_Deg,Wind_Gust,Clouds,Avg_Clouds,Pop,Rain,Rain_Fall,Snow,UVI
0,33,Aix en Provence,Renaissance Aix-en-Provence Hotel,1692,Aix-en-Provence,0.6 km from downtown,Located within a 10-minute walk of the center ...,"2 swimming pools, Private Parking, Spa, Free W...",Aix-en-Provence,43.52624,...,1.96,302,2.68,0,54.25,0.00,NaN,0.595,NaN,1.97
1,33,Aix en Provence,Renaissance Aix-en-Provence Hotel,1692,Aix-en-Provence,0.6 km from downtown,Located within a 10-minute walk of the center ...,"2 swimming pools, Private Parking, Spa, Free W...",Aix-en-Provence,43.52624,...,2.31,234,2.17,41,54.25,0.00,NaN,0.595,NaN,1.79
2,33,Aix en Provence,Renaissance Aix-en-Provence Hotel,1692,Aix-en-Provence,0.6 km from downtown,Located within a 10-minute walk of the center ...,"2 swimming pools, Private Parking, Spa, Free W...",Aix-en-Provence,43.52624,...,1.72,290,2.02,0,54.25,0.00,NaN,0.595,NaN,1.88
3,33,Aix en Provence,Renaissance Aix-en-Provence Hotel,1692,Aix-en-Provence,0.6 km from downtown,Located within a 10-minute walk of the center ...,"2 swimming pools, Private Parking, Spa, Free W...",Aix-en-Provence,43.52624,...,2.60,245,2.35,0,54.25,0.00,NaN,0.595,NaN,1.97
4,33,Aix en Provence,Renaissance Aix-en-Provence Hotel,1692,Aix-en-Provence,0.6 km from downtown,Located within a 10-minute walk of the center ...,"2 swimming pools, Private Parking, Spa, Free W...",Aix-en-Provence,43.52624,...,7.53,107,12.79,96,54.25,0.02,NaN,0.595,NaN,1.74


### Load Merged Dataset

In [ ]:
# Load merged cleaned dataframe into the DW
hotel_weather_data.to_sql(
    "hotel_weather_data",   # table name in DB
    engine,
    if_exists="replace",  
    index=False          
)

print("✅ Data loaded successfully into Postgres!")


2025-08-19 12:12:15,245 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-08-19 12:12:15,247 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-19 12:12:15,280 INFO sqlalchemy.engine.Engine select current_schema()
2025-08-19 12:12:15,281 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-19 12:12:15,307 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-08-19 12:12:15,308 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-19 12:12:15,330 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-19 12:12:15,355 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

#### Sanity Check

In [45]:
# Row count 
n_db = pd.read_sql("SELECT COUNT(*) AS n FROM public.hotel_weather_data", engine)["n"][0]
print("Rows in DB:", n_db, "| Rows in local df:", len(hotel_weather_data))

# Peek a few rows
print(pd.read_sql("SELECT * FROM public.hotel_weather_data LIMIT 5", engine))

# Columns & types
print(pd.read_sql("""
  SELECT column_name, data_type
  FROM information_schema.columns
  WHERE table_schema='public' AND table_name='hotel_weather_data'
  ORDER BY ordinal_position
""", engine))


2025-08-19 12:26:00,788 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-19 12:26:00,793 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-08-19 12:26:00,829 INFO sqlalchemy.engine.Engine [cached since 825.5s ago] {'table_name': 'SELECT COUNT(*) AS n FROM public.hotel_weather_data', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-08-19 12:26:01,012 INFO sqlalchemy.engine.Engine SELECT COUNT(*) AS n FROM public.hotel_weather_data
2025-08-19 12:26:01,014 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-19 12

#### Checking connection with datawarehouse

##### Connect to the default DB and create the kayak database

In [ ]:
# pip install sqlalchemy psycopg2-binary pandas s3fs boto3

from sqlalchemy import create_engine, text

DBUSER   = "jedha_user"
#DBNAME   = "postgres"
PORT     = 5432
TARGET_DB = "jedha-kayak"    

# connect to the default 'postgres' database
admin_engine = create_engine(
    f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}:{PORT}/postgres?sslmode=require",
    isolation_level="AUTOCOMMIT",   # CREATE DATABASE needs autocommit
    pool_pre_ping=True,
)

with admin_engine.connect() as conn:
    # list databases
    rows = conn.execute(text("SELECT datname FROM pg_database")).fetchall()
    print("Existing DBs:", [r[0] for r in rows])

    # create your DB if missing (the hyphen requires quotes)
    if TARGET_DB not in [r[0] for r in rows]:
        conn.execute(text(f'CREATE DATABASE "{TARGET_DB}" ENCODING \'UTF8\' TEMPLATE template1'))
        print(f'Created database "{TARGET_DB}".')


Existing DBs: ['template0', 'template1', 'postgres', 'rdsadmin']
Created database "jedha-kayak".


#### Reconnect to new DB and ping

In [5]:
from sqlalchemy import create_engine, text

DBUSER, DBPASS = "jedha_user", "Jedha2025!"
DBHOST, PORT   = "jedha-kayak.cxsio8oio4r1.eu-west-3.rds.amazonaws.com", 5432
DBNAME         = "jedha-kayak"  # now exists

engine = create_engine(
    f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}:{PORT}/{DBNAME}?sslmode=require",
    pool_pre_ping=True
)

with engine.connect() as conn:
    print(conn.execute(text("SELECT current_database(), 1")).fetchone())

('jedha-kayak', 1)
